### load necessary modules

In [1]:
import uproot
import awkward as ak
import vector
vector.register_awkward()
from matplotlib import pyplot as plt
import mplhep as hep
hep.style.use("CMS")
import math
import itertools
import os
from typing import Dict, List, Tuple
from numpy.typing import ArrayLike
import numpy as np
import correctionlib
import awkward as ak
import fastjet
from coffea.nanoevents.methods import vector
from coffea import nanoevents
from coffea import processor
from coffea.nanoevents.methods import candidate
from coffea.analysis_tools import Weights, PackedSelection
from hist import Hist
ak.behavior.update(vector.behavior)
import sys
sys.path.append('/home/pku/zhaoyz/Higgs/LundReweighting/utils')
from LundReweighter import *
# import pandas as pd

Welcome to JupyROOT 6.28/00


### Load Ntuple files, and actually some selection is done in the Ntuple step 

In [2]:
events = nanoevents.NanoEventsFactory.from_root(
        "/data/bond/zhaoyz/Ntuple/V3/2018/Merged/ttbar_validation_final/TTToSemiLeptonic.root",
        schemaclass=nanoevents.NanoAODSchema,
    ).events()

/home/pku/zhaoyz/anaconda3/envs/lpr/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_genPartIdx => GenPart
  warnings.warn(
/home/pku/zhaoyz/anaconda3/envs/lpr/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_jetIdx => Jet
  warnings.warn(
/home/pku/zhaoyz/anaconda3/envs/lpr/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_photonIdx => Photon
  warnings.warn(
/home/pku/zhaoyz/anaconda3/envs/lpr/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:208: RuntimeWarning: Missing cross-reference target for FatJet_genJetAK8Idx => GenJetAK8
  warnings.warn(
/home/pku/zhaoyz/anaconda3/envs/lpr/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for FsrPhoton_muonIdx => Muon
  warning

In [3]:
events[0].PFCands.pdgId

<Array [211, -211, -211, -211, ... 130, 22, 22] type='276 * int32[parameters={"_...'>

In [4]:
frac = 1
n_use = int(frac*len(events))
events_1 = events[:n_use]

In [5]:
#quick look at the fields of loaded files
events_1.fields

['etagenq5f',
 'phigenzf',
 'phigenq5f',
 'phigenq3l',
 'ST',
 'Flag',
 'puWeightUp',
 'OtherPV',
 'massgenq3f',
 'etagenq5l',
 'PFCands',
 'LHEWeight',
 'massgenq2f',
 'genWeight',
 'phigenq3f',
 'massgenwl',
 'massgenwf',
 'puWeight',
 'GenFatJetSVs',
 'DeepMETResolutionTune',
 'AK15PuppiSubJet',
 'Nj8',
 'puWeightDown',
 'SubJet',
 'mothergenq2f',
 'run',
 'RawPuppiMET',
 'GenVtx',
 'DeepMETResponseTune',
 'GenPart',
 'massgenq1l',
 'genH',
 'phigenq1l',
 'etagenq1f',
 'etagenq4l',
 'mothergenq4f',
 'etagenq1l',
 'CorrT1METJet',
 'MET',
 'etagenzl',
 'Jet',
 'taggenwl',
 'taggenzl',
 'massgengl',
 'massgengf',
 'fixedGridRhoFastjetAll',
 'massgenzl',
 'massgenq5f',
 'SV',
 'ak4jet',
 'HLT',
 'phigenq5l',
 'PuppiMET',
 'ptgenq2l',
 'etagenzf',
 'FatJet',
 'phigenq2l',
 'massgenq3l',
 'massgenq5l',
 'etagengl',
 'fixedGridRhoFastjetCentralNeutral',
 'phigengl',
 'ptgenq4l',
 'LHE',
 'gent',
 'mothergengf',
 'FatJetPFCands',
 'GenCands',
 'genantit',
 'CaloMET',
 'm',
 'mothergenq5f',


In [6]:
del events

In [7]:
events_1

<NanoEventsArray [<event 1:142148:142147035>, ... ] type='2758516 * event'>

### Define necessary functions to run the selection

In [8]:
#pad array with given value
def pad_val(
    arr: ak.Array,
    target: int,
    value: float, #value can also be Bool variable 
    axis: int = 0,
    to_numpy: bool = True,
    clip: bool = True,
):
    """
    pads awkward array up to ``target`` index along axis ``axis`` with value ``value``,
    optionally converts to numpy array
    """
    padded_arr = ak.fill_none(ak.pad_none(arr, target, axis=axis, clip=clip), value, axis=axis)
    # pad_none will fill the array to target length with "None" for dedicated axis
    # "clip" means cut the array to the target length or not
    # fill_none will replace "None" value to some value
    return padded_arr.to_numpy() if to_numpy else padded_arr

def add_selection(
    name: str,
    sel: np.ndarray,
    selection: PackedSelection,
    cutflow: dict = None,
    isData: bool = False,
    signGenWeights: ak.Array = None,
):
    """adds selection to PackedSelection object and the cutflow dictionary"""
    selection.add(name, sel)
    if cutflow is not None: #only add to cutflow dictionary if cutflow is not None
        cutflow[name] = (
            np.sum(selection.all(*selection.names))
            if isData
            # add up sign of genWeights for MC
            else np.sum(signGenWeights[selection.all(*selection.names)])
        )


In [9]:
#pre-selection:
#1.Leading jet pT > 400GeV, maximum jet mass > 50GeV
#2.Require 2 or 3 AK8 jet with pT > 200GeV
#3.Veto (mini-)Isolated leptons
isData = False
signGenWeights = None if isData else np.sign(events_1["genWeight"]) #get genWeight sign, because only the sign matters
n_events = len(events_1) if isData else int(np.sum(signGenWeights)) #events number for MC events should be the sum of "sign"
selection = PackedSelection() #initialize a new object

cutflow = {}
# cutflow["all"] = len(events) #shouldn't be n_events?
cutflow["all"] = n_events
preselection_cut_vals = {"pt": 200, "msd": 20, "leading_pt":400,"maximum_mass":50}
num_jets = 2

# fatjets = corrections.get_jec_jets(events, "2018")
fatjets = events_1.FatJet

preselection_cut_1 = pad_val(
        ( ak.max(events_1.FatJet.pt, axis = 1) > preselection_cut_vals["leading_pt"])
        * (ak.max(events_1.FatJet.msoftdrop, axis = 1) > preselection_cut_vals["maximum_mass"]), #mass and pT cut of each jet in event
        len(events_1), #pad to num_jets length
        False,  #pad with value False
        )
# finally with the length of events number, "1" for all jets are pT > pT_cut and mass > mass_cut
 # N.B. here clip always = True

add_selection(
    "leading pT and maximum mass", #string name
    preselection_cut_1.astype(bool), #selection content
    selection, #PackedSelection object
    cutflow, #cut-flow dict, storing events number after each cut
    isData,
    signGenWeights,#sum the signGenWeights for events which pass the selection
)



preselection_cut_2 = np.prod(
    pad_val(
        (events_1.FatJet.pt > preselection_cut_vals["pt"]),
        # * (events.FatJet.msoftdrop > preselection_cut_vals["msd"]), #mass and pT cut of each jet in event
        num_jets, #pad to num_jets length
        False,  #pad with value False
        axis=1, #pad to axis=1
    ),
    axis=1,
)# finally with the length of events number, "1" for all jets are pT > pT_cut and mass > mass_cut
 # N.B. here clip always = True

add_selection(
    "at least 2 AK8 jet with pT >200GeV", #string name
    preselection_cut_2.astype(bool), #selection content
    selection, #PackedSelection object
    cutflow, #cut-flow dict, storing events number after each cut
    isData,
    signGenWeights,#sum the signGenWeights for events which pass the selection
)

preselection_cut_3 = pad_val(
        (ak.num(events_1.FatJet.pt) == 2) | (ak.num(events_1.FatJet.pt) == 3) , #mass and pT cut of each jet in event
        len(events_1), #pad to num_jets length
        False,  #pad with value False
        )

add_selection(
    "2 or 3 AK8 jet", #string name
    preselection_cut_3.astype(bool), #selection content
    selection, #PackedSelection object
    cutflow, #cut-flow dict, storing events number after each cut
    isData,
    signGenWeights,#sum the signGenWeights for events which pass the selection
)


In [10]:
cutflow

{'all': 2688878,
 'leading pT and maximum mass': 2637411.0,
 'at least 2 AK8 jet with pT >200GeV': 2637249.0,
 '2 or 3 AK8 jet': 2513907.0}

In [11]:
events_2 = events_1[selection.all(*selection.names)]

In [12]:
events_2

<NanoEventsArray [<event 1:142148:142147035>, ... ] type='2581743 * event'>

In [13]:
del events_1

In [14]:
events_3 = events_2
del events_2

### Preselection done here, but we still have to do Top enrich selection

In [15]:
#find leading tagger score jet, denote it as leading_fatjet
eventsScoreFatjet = events_3
tagger_scores = eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWev0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWev1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWmv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWmv1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq2c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq2c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauev0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauev1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauhv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauhv1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtaumv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtaumv1c
#get maximum tagger score jet index
max_tagger_indices = (tagger_scores == np.max(tagger_scores, axis=1))
fatjets = events_3.FatJet
leading_fatjet = fatjets[max_tagger_indices][:,0]

### Collect inclusive tight b jets

In [16]:
btagDeepFlavB_WP = {
        "bWPloose"  : 0.0490,
        "bWPmedium" : 0.2783,
        "bWPtight"  : 0.7100,
    } #2018 btagDeepFlavB working point

In [17]:
#top enriched-selection:
#1. 125GeV<Mja<225GeV
#2. MET_pt/PTja>0.4
#3. PTja>500GeV
#4. Tight WP inclusive b jets >= 1

isData = False
signGenWeights = None if isData else np.sign(events_3["genWeight"]) #get genWeight sign, because only the sign matters
n_events = len(events_3) if isData else int(np.sum(signGenWeights)) #events number for MC events should be the sum of "sign"
selection = PackedSelection() #initialize a new object

cutflow = {}
# cutflow["all"] = len(events) #shouldn't be n_events?
cutflow["all"] = n_events
muon_enriched_cut_vals = {"pt_min": 400, "MET_pt_min":0.4, "msd_min": 125, "msd_max":225}

muon_enriched_cut_1 = pad_val(
        ( leading_fatjet.msoftdrop >= muon_enriched_cut_vals["msd_min"])
        * ( leading_fatjet.msoftdrop <= muon_enriched_cut_vals["msd_max"])
        * ( leading_fatjet.pt >= muon_enriched_cut_vals["pt_min"]) 
        * ( events_3.MET.pt/leading_fatjet.pt >= 0.4), #mass and pT cut of each jet in event
        len(events_3), #pad to num_jets length
        False,  #pad with value False
        )

add_selection(
    "pt mass MET selection", #string name
    muon_enriched_cut_1.astype(bool), #selection content
    selection, #PackedSelection object
    cutflow, #cut-flow dict, storing events number after each cut
    isData,
    signGenWeights,#sum the signGenWeights for events which pass the selection
)

nb_t_inclusive_collections = (events_3.Jet.btagDeepFlavB >= btagDeepFlavB_WP["bWPtight"])
nb_t_inclusive = np.sum(nb_t_inclusive_collections, axis = 1)

muon_enriched_cut_2 = pad_val(
    (nb_t_inclusive >= 1),
    len(events_3),
    False,
)# finally with the length of events number, "1" for all jets are pT > pT_cut and mass > mass_cut
 # N.B. here clip always = True

add_selection(
    "at least 1 inclusive tight b jets", #string name
    muon_enriched_cut_2.astype(bool), #selection content
    selection, #PackedSelection object
    cutflow, #cut-flow dict, storing events number after each cut
    isData,
    signGenWeights,#sum the signGenWeights for events which pass the selection
)

delta_phi = np.subtract(events_3.MET.phi, leading_fatjet.phi)
delta_phi = np.where(delta_phi > np.pi, delta_phi - 2*np.pi, delta_phi)
delta_phi = np.where(delta_phi < -np.pi, delta_phi + 2*np.pi, delta_phi)
delta_phi = np.abs(delta_phi)


dphi_cut = pad_val(
    (delta_phi >= 2.0),
    len(events_3),
    False,
)# finally with the length of events number, "1" for all jets are pT > pT_cut and mass > mass_cut
 # N.B. here clip always = True
 
add_selection(
    "dphi cut", #string name
    dphi_cut.astype(bool), #selection content
    selection, #PackedSelection object
    cutflow, #cut-flow dict, storing events number after each cut
    isData,
    signGenWeights,#sum the signGenWeights for events which pass the selection
)

In [18]:
cutflow

{'all': 2513907,
 'pt mass MET selection': 139264.0,
 'at least 1 inclusive tight b jets': 117859.0,
 'dphi cut': 100239.0}

In [19]:
events_4 = events_3[selection.all(*selection.names)]
del events_3

### Start to run LP stuff

In [20]:
selection = PackedSelection() #initialize a new object
isData = False
signGenWeights = None if isData else np.sign(events_4["genWeight"]) #get genWeight sign, because only the sign matters
n_events = len(events_4) if isData else int(np.sum(signGenWeights)) #events number for MC events should be the sum of "sign"
selection = PackedSelection() #initialize a new object

cutflow = {}
# cutflow["all"] = len(events) #shouldn't be n_events?
cutflow["all"] = n_events

#require WW decaying to 4q, because we want to calibrate H3q4q jet at first stage
d_PDGID = 1
u_PDGID = 2
s_PDGID = 3
c_PDGID = 4
b_PDGID = 5
g_PDGID = 21
TOP_PDGID = 6

ELE_PDGID = 11
vELE_PDGID = 12
MU_PDGID = 13
vMU_PDGID = 14
TAU_PDGID = 15
vTAU_PDGID = 16

Z_PDGID = 23
W_PDGID = 24
HIGGS_PDGID = 25
Y_PDGID = 35

b_PDGIDS = [511, 521, 523]

GRAV_PDGID = 39

GEN_FLAGS = ["fromHardProcess", "isLastCopy"]

FILL_NONE_VALUE = -99999
PAD_VAL = -99999

skim_vars = {
    "eta": "Eta",
    "phi": "Phi",
    "mass": "Mass",
    "pt": "Pt",
}

### start to match top, b and W

In [21]:
print(len(events_4))

107157


In [22]:
#find gen-level top quark
tops = events_4.GenPart[
        (abs(events_4.GenPart.pdgId) == TOP_PDGID) * events_4.GenPart.hasFlags(GEN_FLAGS)
]


In [23]:
#new leading tagger score jet, denote it as leading_fatjet
eventsScoreFatjet = events_4
tagger_scores = eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWev0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWev1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWmv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWmv1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq2c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq2c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauev0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauev1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauhv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauhv1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtaumv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtaumv1c
#get maximum tagger score jet index
max_tagger_indices = (tagger_scores == np.max(tagger_scores, axis=1))
fatjets = events_4.FatJet
leading_fatjet = fatjets[max_tagger_indices][:,0]

In [24]:
#find top children, refer to:https://github.com/rkansal47/HHbbVV/blob/2fc4110669081d565e115d32486c7d555db5bac7/src/HHbbVV/processors/GenSelection.py#L888
tops_children = tops.distinctChildren
tops_children = tops_children[tops_children.hasFlags(GEN_FLAGS)]
#may take some time since it's recursive search

In [25]:
#get gen-level W informatino
ws = ak.flatten(tops_children[np.abs(tops_children.pdgId) == W_PDGID], axis=2)

In [26]:
len(ws)

107157

In [27]:
# get hadronic W and top
had_top_sel = np.all(np.abs(ws.children.pdgId) <= 5, axis=2)
had_ws = ak.flatten(ws[had_top_sel])
had_ws_children = had_ws.children
had_tops = ak.flatten(tops[had_top_sel])

In [28]:
# get leptonic W and top(since we are using ttbar_semilep sample)
lep_top_sel = np.all(np.abs(ws.children.pdgId) >= 11, axis=2)
lep_ws = ak.flatten(ws[lep_top_sel])
lep_ws_children = lep_ws.children
lep_tops = ak.flatten(tops[lep_top_sel])

In [29]:
lep_leps = lep_ws_children[(np.abs(lep_ws_children.pdgId) == 11) | (np.abs(lep_ws_children.pdgId) == 13) | (np.abs(lep_ws_children.pdgId) == 15)] #select lepton but not nu, it's always the first element

In [30]:
# check for b's from top
had_top_children = ak.flatten(tops_children[had_top_sel], axis=1)
had_bs = had_top_children[np.abs(had_top_children.pdgId) == 5]
# add_selection("top_has_bs", np.any(had_bs.pdgId, axis=1), *selection_args)

In [31]:
# b quark and hadronic decay W boson from the top
# gen_quarks = ak.concatenate([had_bs[:, :1], had_ws_children[:, :2]], axis=1)
had_bs.eta[:, :1]
gen_quarks_eta = ak.concatenate([had_bs.eta[:, :1], had_ws_children.eta[:, :2]], axis=1)
gen_quarks_phi = ak.concatenate([had_bs.phi[:, :1], had_ws_children.phi[:, :2]], axis=1)


In [32]:
# similarly, collect lepton and hadronic decay W boson from the top
gen_quarks_lepton_eta = ak.concatenate([lep_leps.eta[:,:1], had_ws_children.eta[:, :2]], axis=1)
gen_quarks_lepton_phi = ak.concatenate([lep_leps.phi[:,:1], had_ws_children.phi[:, :2]], axis=1)

In [33]:
deltaR = 0.8
had_w_jet_match = ak.fill_none(
    ak.all(had_ws_children.delta_r(leading_fatjet) < deltaR, axis=1), False
)

In [34]:
deltaR = 0.8
lep_jet_match = ak.flatten(
    pad_val(
        ak.fill_none(lep_leps.delta_r(leading_fatjet) < deltaR, [], axis=0),
        1,
        False,
        axis=1,
        to_numpy=False,
    )
)

In [35]:
deltaR = 0.8
had_b_jet_match = ak.flatten(
    pad_val(
        ak.fill_none(had_bs.delta_r(leading_fatjet) < deltaR, [], axis=0),
        1,
        False,
        axis=1,
        to_numpy=False,
    )
)
top_match_dict = {
    "top_matched": had_w_jet_match * had_b_jet_match * ~lep_jet_match,
    "w_matched": had_w_jet_match * ~had_b_jet_match * ~lep_jet_match,
    "tlqq_matched" : had_w_jet_match * lep_jet_match,
    "unmatched": ~had_w_jet_match,
}
top_match_dict = {key: val.to_numpy().astype(int) for key, val in top_match_dict.items()}


### store the PFCands, GenEtaPhi, Higgs candidate AK8 jet 4-vector information for Lund Plane use

In [36]:
#only select events whose jet_a is top jets
events_after_cut = events_4[top_match_dict["top_matched"].astype(bool)]
events_after_cut

<NanoEventsArray [<event 1:143567:143566774>, ... ] type='62517 * event'>

### four vector for HWW jet

In [37]:
HWWJets = leading_fatjet[top_match_dict["top_matched"].astype(bool)]

In [38]:
# four vector for HWW jet
higgs_jet_4vec = np.array(np.stack((np.array(HWWJets.pt), np.array(HWWJets.eta),np.array(HWWJets.phi),np.array(HWWJets.mass)), axis=1))
higgs_jet_4vec[1]


array([615.       ,  -0.9633789,   0.7246094, 187.       ], dtype=float32)

In [39]:
len(higgs_jet_4vec)

62517

### Next eta-phi for 4 quarks

In [40]:
#here only the quarks from tbqq matched are used
eta = gen_quarks_eta[top_match_dict["top_matched"].astype(bool)].to_numpy()
phi = gen_quarks_phi[top_match_dict["top_matched"].astype(bool)].to_numpy()

In [41]:
gen_parts_eta_phi_bqq = np.array(np.dstack((eta,phi)))
# can do test like : gen_parts_eta_phi[:2]

### Get FatJetPFCands 4-vector, up to 150 length to suit the input of Oz's function

In [42]:
events_final = events_after_cut
events_final

<NanoEventsArray [<event 1:143567:143566774>, ... ] type='62517 * event'>

In [43]:
# HWW_jet_idx = ak.argmax(tagger_scores[], axis=1)
eventsScoreFatjet = events_final
tagger_scores = eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWev0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWev1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWmv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWmv1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWq2c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWqq2c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauev0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauev1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauhv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtauhv1c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtaumv0c + eventsScoreFatjet.FatJet.inclParTMDV1_probHWqqWtaumv1c
#get maximum tagger score jet index
HWW_jet_idx = ak.argmax(tagger_scores, axis=1)


In [44]:
HWW_jet_idx

<Array [0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0] type='62517 * ?int64'>

In [45]:
# first get the jet_idx HWW jet, each event has one jet_idx
# HWW_match = WWdr <= match_dR #FatJetIdx in each event, which is real HWW jet
# HWW_match_padded = pad_val(HWW_match,3,False,1,True) #pad the array with False value
# HWW_jet_idx = np.argmax(HWW_match_padded,axis = 1) #the jet index in each jet which is true HWW jet
# then get all the FatJetPFCands according to the jet_idx, and get PF_idx
HWW_FatJetPFCands = (events_final.FatJetPFCands.jetIdx == HWW_jet_idx)
HWW_FatJetPFCands_pFCandsIdx = events_final.FatJetPFCands.pFCandsIdx[HWW_FatJetPFCands]
# at last, get PFCands 4-vector according to the PF_idx in last step


In [46]:
HWW_FatJetPFCands[0]

<Array [True, True, True, ... False, False] type='101 * bool'>

In [47]:
HWW_FatJetPFCands_pFCandsIdx[0]

<Array [14, 17, 18, 25, ... 190, 192, 193, 199] type='68 * int32[parameters={"__...'>

In [48]:
pt_array =   ak.Array(events_final.PFCands.pt)
eta_array =  ak.Array(events_final.PFCands.eta)
phi_array =  ak.Array(events_final.PFCands.phi)
mass_array = ak.Array(events_final.PFCands.mass)

In [49]:
selected_pt =  pt_array[HWW_FatJetPFCands_pFCandsIdx]
selected_eta = eta_array[HWW_FatJetPFCands_pFCandsIdx]
selected_phi = phi_array[HWW_FatJetPFCands_pFCandsIdx]
selected_mass = mass_array[HWW_FatJetPFCands_pFCandsIdx]

In [50]:
selected_pt_padded = pad_val(selected_pt,150,0,1,True)
selected_eta_padded = pad_val(selected_eta,150,0,1,True)
selected_phi_padded = pad_val(selected_phi,150,0,1,True)
selected_mass_padded = pad_val(selected_mass,150,0,1,True)

In [51]:
#Construct (px,py,pz,E) using (pt,eta,phi,mass) information as the input
pf_cands_px = selected_pt_padded * np.cos(selected_phi_padded)
pf_cands_py = selected_pt_padded * np.sin(selected_phi_padded)
pf_cands_pz = selected_pt_padded * np.sinh(selected_eta_padded)
pf_cands_E = np.sqrt(pf_cands_px**2 + pf_cands_py**2 + pf_cands_pz**2 + selected_mass_padded**2)

In [52]:
pf_cands_pxpypzE = np.dstack((pf_cands_px,pf_cands_py,pf_cands_pz,pf_cands_E))

In [53]:
pf_cands_4vec = np.dstack((selected_pt_padded,selected_eta_padded,selected_phi_padded,selected_mass_padded))

### Next compute tagger score

In [54]:
tagger_scores = (leading_fatjet.inclParTMDV1_probHWqqWev0c + leading_fatjet.inclParTMDV1_probHWqqWev1c + leading_fatjet.inclParTMDV1_probHWqqWmv0c + leading_fatjet.inclParTMDV1_probHWqqWmv1c + leading_fatjet.inclParTMDV1_probHWqqWq0c + leading_fatjet.inclParTMDV1_probHWqqWq1c + leading_fatjet.inclParTMDV1_probHWqqWq2c + leading_fatjet.inclParTMDV1_probHWqqWqq0c + leading_fatjet.inclParTMDV1_probHWqqWqq1c + leading_fatjet.inclParTMDV1_probHWqqWqq2c + leading_fatjet.inclParTMDV1_probHWqqWtauev0c + leading_fatjet.inclParTMDV1_probHWqqWtauev1c + leading_fatjet.inclParTMDV1_probHWqqWtauhv0c + leading_fatjet.inclParTMDV1_probHWqqWtauhv1c + leading_fatjet.inclParTMDV1_probHWqqWtaumv0c + leading_fatjet.inclParTMDV1_probHWqqWtaumv1c)/(ak.ones_like(leading_fatjet.inclParTMDV1_probHWqqWev0c) - leading_fatjet.inclParTMDV1_probTopbWtauev - leading_fatjet.inclParTMDV1_probTopbWq0c - leading_fatjet.inclParTMDV1_probTopbWmv - leading_fatjet.inclParTMDV1_probTopbWev - leading_fatjet.inclParTMDV1_probTopbWqq0c - leading_fatjet.inclParTMDV1_probTopbWqq1c - leading_fatjet.inclParTMDV1_probTopbWtauhv - leading_fatjet.inclParTMDV1_probTopbWq1c -leading_fatjet.inclParTMDV1_probTopbWtaumv)
HWWJets_tagger_score = tagger_scores[top_match_dict["top_matched"].astype(bool)]

In [55]:
HWWJets_tagger_score

<Array [0.055, 0.263, 0.0344, ... 0.162, 0.147] type='62517 * ?float32'>

### Calculate SFs and weights

In [58]:
import sys, os
sys.path.insert(0, '')
sys.path.append("/home/pku/zhaoyz/Higgs/LundReweighting")
from utils.LundReweighter import *
from utils.Utils import *
""" An example how to use the Lund Plane reweighting  code """

######################## Setup 

#Input file 
fname = "/home/pku/zhaoyz/Higgs/LundReweighting/data/example_signal.h5"
#File containing data/MC Lund Plane ratio
f_ratio_name = '/home/pku/zhaoyz/Higgs/LundReweighting/data/ratio_2018.root'

f_sig = h5py.File(fname, "r")
f_ratio = ROOT.TFile.Open(f_ratio_name)

#Class to help read input dataset, "Dataset" class defined in Utils.py
# d = Dataset(f_sig, dtype = 1)
# d.compute_obs() #there may be some dedicated variables inside the .h5 file

#The cut we will compute a SF for 'tau21 < 0.34'
tag_obs = 'tau21'
score_thresh = 0.975


#nominal data/MC Lund plane ratio (3d histogram)
h_ratio = f_ratio.Get("ratio_nom")
#systematic variations
h_ratio_sys_up = f_ratio.Get("ratio_sys_tot_up")
h_ratio_sys_down = f_ratio.Get("ratio_sys_tot_down")
#MC ratio of b to light quarks
b_light_ratio = f_ratio.Get("h_bl_ratio")


#directory of pt extrapolation fits
f_ratio.cd('pt_extrap')
rdir = ROOT.gDirectory #get the present working directory and give it to rdir

#Main class for reweighting utilities
LP_rw = LundReweighter(pt_extrap_dir = rdir)

max_evts = 50000
tagger_cut_value = [0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95]
for value_i in tagger_cut_value:
    # score = getattr(d, tag_obs)[:max_evts]
    score_cut = ((HWWJets_tagger_score >= value_i ) & (HWWJets_tagger_score <= (value_i + 0.5)))
    # score_cut = ((HWWJets_tagger_score >= 0.9) & (HWWJets_tagger_score <= 0.95))
    # score_cut = (HWWJets_tagger_score > 0.95)
    score_cut = score_cut[:max_evts]


    #Number of toys for statistical and pt extrapolation uncertainties
    nToys = 100
    #Noise vectors used to to generate the toys
    #NOTE the same vector has to be used for the whole sample/signal file for the toys to be consistent 
    rand_noise = np.random.normal(size = (nToys, h_ratio.GetNbinsX(), h_ratio.GetNbinsY(), h_ratio.GetNbinsZ()))
    pt_rand_noise = np.random.normal(size = (nToys, h_ratio.GetNbinsY(), h_ratio.GetNbinsZ(), 3))


    ################### Compute reweighting factors

    #PF candidates in the AK8 jet
    # pf_cands = d.get_masked("jet1_PFCands").astype(np.float64)[:max_evts]
    pf_cands = pf_cands_pxpypzE[:max_evts]
    #Generator level quarks from hard process

    # gen_parts = d.get_masked('gen_info')[:max_evts]
    gen_parts_eta_phi = gen_parts_eta_phi_bqq[:max_evts]
    # gen_parts_pdg_ids = gen_parts[:,:,3]

    B_PDG_ID = 5

    # ak8_jets = d.get_masked('jet_kinematics')[:max_evts][:,2:6].astype(np.float64)
    ak8_jets = higgs_jet_4vec[:max_evts]

    #Nominal event weights of the MC, assume every event is weight '1' for this example
    weights_nom = np.ones(max_evts)

    LP_weights = []
    LP_weights_sys_up = []
    LP_weights_sys_down = []
    stat_smeared_weights = []
    pt_smeared_weights = []
    b_weights_up = []
    b_weights_down = []
    bad_matches = []


    for i,cands in enumerate(pf_cands):
        # if i == 4: break
        # print("now processing:",i)
        #Get the subjets, splittings and checking matching based on PF candidates in the jet and gen-level quarks
        subjets, splittings, bad_match, deltaRs = LP_rw.get_splittings_and_matching(cands, gen_parts_eta_phi[i], ak8_jets[i])
        # print(bad_match)
        # print(deltaRs)
        #Gets the nominal LP reweighting factor for this event and statistical + pt extrapolation toys
        LP_weight, stat_smeared_weight, pt_smeared_weight = LP_rw.reweight_lund_plane(h_rw = h_ratio, subjets = subjets, splittings = splittings,
                rand_noise = rand_noise, pt_rand_noise = pt_rand_noise, )
        #Now get systematic variations
        LP_weight_sys_up,_,_ = LP_rw.reweight_lund_plane(h_rw = h_ratio_sys_up, subjets = subjets, splittings = splittings)
        LP_weight_sys_down,_,_ = LP_rw.reweight_lund_plane(h_rw = h_ratio_sys_down, subjets = subjets, splittings = splittings)

        LP_weights.append(LP_weight)
        stat_smeared_weights.append(stat_smeared_weight)
        pt_smeared_weights.append(pt_smeared_weight)

        LP_weights_sys_up.append(LP_weight_sys_up)
        LP_weights_sys_down.append(LP_weight_sys_down)
        bad_matches.append(bad_match)



    ############### Normalize weights to preserve normalization of the MC sample

    #The nominal Lund Plane correction event weights
    LP_weights = LP_rw.normalize_weights(LP_weights) * weights_nom 

    #Toy variations for stat and pt uncertainties
    stat_smeared_weights = LP_rw.normalize_weights(stat_smeared_weights) * weights_nom.reshape(max_evts, 1)
    pt_smeared_weights = LP_rw.normalize_weights(pt_smeared_weights) * weights_nom.reshape(max_evts,1)

    #Systematic up/down variations
    LP_weights_sys_up = LP_rw.normalize_weights(LP_weights_sys_up) * weights_nom
    LP_weights_sys_down = LP_rw.normalize_weights(LP_weights_sys_down) * weights_nom

    ############### Compute efficiences and uncertainties


    #Efficiency of the cut in nominal MC
    eff_nom = np.average(score_cut, weights = weights_nom) #TODO

    #Efficiency of the cut after the Lund Plane reweighting
    eff_rw = np.average(score_cut, weights = LP_weights)

    #Nominal 'scale factor'
    SF = eff_rw / eff_nom

    print("Nominal efficiency %.3f, Corrected efficiency %.3f, SF (corrected / nom) %.3f" % (eff_nom, eff_rw, SF))

    #NOTE, better to use corrected efficiency computed separately for each sample rather than a single 'SF'


    #Compute efficiency for each of the stat/pt toys
    eff_toys = []
    pt_eff_toys = []
    for i in range(nToys):
        eff = np.average(score_cut, weights = stat_smeared_weights[:,i])
        eff_toys.append(eff)

        eff1 = np.average(score_cut, weights = pt_smeared_weights[:,i])
        pt_eff_toys.append(eff1)

    #Compute stat and pt uncertainty based on variation in the toys
    toys_mean = np.mean(eff_toys)
    toys_std = np.std(eff_toys)
    pt_toys_mean = np.mean(pt_eff_toys)
    pt_toys_std = np.std(pt_eff_toys)

    eff_stat_unc = (abs(toys_mean - eff_rw)  + toys_std) 
    eff_pt_unc = (abs(pt_toys_mean - eff_rw) + pt_toys_std)

    print("Stat variation toys eff. avg %.3f, std dev %.3f" % (toys_mean, toys_std))
    print("Pt variation toys eff. avg %.3f, std dev %.3f" % (pt_toys_mean, pt_toys_std))


    #Compute difference in efficiency due to weight variations as uncertainty
    def get_uncs(score_cut, weights_up, weights_down, eff_baseline):
        eff_up =  np.average(score_cut, weights = weights_up)
        eff_down =  np.average(score_cut, weights = weights_down)

        unc_up = eff_up - eff_baseline
        unc_down = eff_down - eff_baseline 
        return unc_up, unc_down


    #Compute efficiency of systematic variations
    sys_unc_up, sys_unc_down = get_uncs(score_cut, LP_weights_sys_up, LP_weights_sys_down, eff_rw)
    # b_unc_up, b_unc_down = get_uncs(score_cut, b_weights_up, b_weights_down, eff_rw)


    #matching uncertainty, taken as a fractional uncertainty on efficiency
    bad_match_frac = np.mean(bad_matches)
    bad_match_unc = bad_match_frac * eff_rw


    ############ Results
    print("\n\nCalibrated efficiency  is %.3f +/- %.3f  (stat) +/- %.3f (pt) +/- %.3f/%.3f (sys)+/- %.3f (matching)  \n\n"  % 
            (eff_rw, eff_stat_unc, eff_pt_unc, sys_unc_up, sys_unc_down, bad_match_unc))

    #next compute the uncertainty about SFs

    #Efficiency of the cut in nominal MC
    eff_nom = np.average(score_cut, weights = weights_nom) #TODO

    #Efficiency of the cut after the Lund Plane reweighting
    eff_rw = np.average(score_cut, weights = LP_weights)

    #Nominal 'scale factor'
    print("Now perform SFs information")
    SF = eff_rw / eff_nom

    print("SF (corrected / nom) %.3f" % (SF))

    #propagate statistical and pt extrapolation uncertainties to SF
    SF_stat_unc = (abs(toys_mean - eff_rw)  + toys_std) /eff_nom
    SF_pt_unc = (abs(pt_toys_mean - eff_rw) + pt_toys_std) /eff_nom

    #propagate systemetic uncertainty to SF
    eff_sys_up =  np.average(score_cut, weights = LP_weights_sys_up)
    eff_sys_down =  np.average(score_cut, weights = LP_weights_sys_down)

    sys_unc_up = abs(eff_rw - eff_sys_up)
    sys_unc_down = abs(eff_rw - eff_sys_down)

    SF_sys_unc_up = sys_unc_up/eff_nom
    SF_sys_unc_down = sys_unc_down/eff_nom

    #calculate bad matching uncertainty directly
    SF_match_unc = bad_match_frac * SF
    print("tagger cut = ",tagger_cut_value)
    print("\n\nSF is %.3f +/-%.3f(stat) +/-%.3f(pt) +%.3f/-%.3f(sys) +/-%.3f(match) \n\n"  % (SF, SF_stat_unc, SF_pt_unc, sys_unc_up, sys_unc_down, SF_match_unc))
    print("total unc. = ",np.sqrt(SF_stat_unc **2 + SF_pt_unc**2 + sys_unc_up**2 + SF_match_unc**2 ))
    print("*******\n")

f_ratio.Close()


Nominal efficiency 0.258, Corrected efficiency 0.268, SF (corrected / nom) 1.039
Stat variation toys eff. avg 0.266, std dev 0.005
Pt variation toys eff. avg 0.268, std dev 0.001


Calibrated efficiency  is 0.268 +/- 0.007  (stat) +/- 0.001 (pt) +/- -0.008/0.009 (sys)+/- 0.047 (matching)  


Now perform SFs information
SF (corrected / nom) 1.039
tagger cut =  [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


SF is 1.039 +/-0.028(stat) +/-0.003(pt) +0.008/-0.009(sys) +/-0.182(match) 


total unc. =  0.1845683994064236
*******

Nominal efficiency 0.224, Corrected efficiency 0.234, SF (corrected / nom) 1.041
Stat variation toys eff. avg 0.233, std dev 0.005
Pt variation toys eff. avg 0.234, std dev 0.001


Calibrated efficiency  is 0.234 +/- 0.006  (stat) +/- 0.001 (pt) +/- -0.008/0.008 (sys)+/- 0.041 (matching)  


Now perform SFs information
SF (corrected / nom) 1.041
tagger cut =  [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


SF is 1.041 +/-0.025(stat) +/-0.003(pt) +0.008/-0.008(sys) +/-

### Make some new plots

In [57]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from cycler import cycler
max_evts = 60000
#implement CMS plot style functions
use_helvet = False ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet['font.sans-serif'] = ['Helvetica', 'Arial']
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)

plt.figure(figsize=(10,10))
ax=plt.gca()
plt.grid()
hep.cms.label(data=False, year="2018", ax=ax, fontname='sans-serif')
%matplotlib inline
#step1: plot 

# plt.hist(eventsEventsID3Prongs4Prongs['HqqqqVsQcdTop'], bins=20, range=(0,1), histtype='step', label='before reweighting',density=True);
# plt.hist(eventsEventsID3Prongs4Prongs['HqqqqVsQcdTop'], bins=20, range=(0,1), histtype='step', label='after reweighting', weights=eventsEventsID3Prongs4Prongs["LP_weight"],density=True);
nbins, x_min, x_max = 20, 0, 1.0
hist_before = bh.Histogram(bh.axis.Regular(nbins, x_min, x_max), storage=bh.storage.Weight())
hist_before.fill(HWWJets_tagger_score[:max_evts])
hist_before_value = hist_before.view().value
hist_before_err = np.sqrt(hist_before.view().variance)
hist_after = bh.Histogram(bh.axis.Regular(nbins, x_min, x_max), storage=bh.storage.Weight())
hist_after.fill(HWWJets_tagger_score[:max_evts],weight=LP_weights[:max_evts])
hist_after_value = hist_after.view().value
hist_after_err = np.sqrt(hist_after.view().variance)
bins = hist_before.axes[0].edges


hep.histplot(hist_before_value,    bins=bins, yerr=hist_before_err, label= 'before Lund Plane reweighting', lw = 2,edges = False, histtype="step")
hep.histplot(hist_after_value,     bins=bins, yerr=hist_after_err,  label= 'after Lund Plane reweighting', lw = 2,edges = False, histtype="step")


plt.legend(loc='upper left',frameon=False,fontsize=20)
y_min,y_max = plt.gca().get_ylim()
plt.text(0.08, 0.83*y_max, "from tt-semileptonic sample,tbqq matched jets", fontsize=20)
# plt.xlabel(r'$H_{qqqq} / (H_{qqqq} + QCD + Top)$')
plt.xlabel(r'$jet_{a}:HWW\ score(without\ P_{top}\ in\ denominator)$', fontsize=20, ha='right', x=1)
plt.ylabel('Events(No xs-weighted)',fontsize=20, ha='right', y=1)
plt.savefig(f"TaggerDistribution_2018_ttbar_tbqq_60000evts.pdf", bbox_inches='tight')
plt.xticks(size=14)
plt.yticks(size=14)
plt.show()


ValueError: spans must have compatible lengths